<a href="https://colab.research.google.com/github/LuisDLCP/MIneria-datos_Examen-final/blob/main/1)segmentacion_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXAMEN FINAL DE MINERIA DE DATOS Y APRENDIZAJE AUTOMATICO 


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
root_path="./drive/MyDrive/Master_Program_CC/AprendizajeAutomatico/Examen_final/RawData/"
!ls {root_path}

data_original1_parte1.mp4  data_original1_parte2.wav  data_test.mp4
data_original1_parte1.wav  data_segmentada	      data_test.wav
data_original1_parte2.mp4  data_test


## 1) Convert video (mp4) to audio (wav) file 

In [ ]:
import subprocess

In [ ]:
video_file = root_path+"data_original1.mp4"
audio_file = root_path+"data_original1.wav"

In [ ]:
#command = f"ffmpeg -i {input_file} -ab 160k -ac 2 -ar 44100 -vn audio.wav"
command = f"ffmpeg -i {video_file} -q:a 0 -map a {audio_file}"

subprocess.call(command, shell=True)

0

## 2. Segmentacion en bloques de 15s de duracion

In [ ]:
pip install pydub 

In [ ]:
from pydub import AudioSegment
archivo_audio = AudioSegment.from_wav(audio_file)

In [ ]:
import os 
to=1000 # ms
period = 15*to # change the value in seconds 
pathSegment = root_path + "data_test/" # output path 
person = ["luis","guiomar","juan-carlos"]

time_total = len(archivo_audio)
time_current = 0*to # change the value in seconds
#time_current = int(time_total/2) # change the value in seconds
file_name_counter = 0
i = 0
numFiles_block = int(time_total/period/len(person))

while time_current < time_total:
  segmento = archivo_audio[time_current:(time_current+period)]
  # File name 
  if i<numFiles_block:
    pathSegment2 = pathSegment + person[0] + "/"
  elif i<2*numFiles_block:
    pathSegment2 = pathSegment + person[1] + "/"
  else:
    pathSegment2 = pathSegment + person[2] + "/"

  out_filename = pathSegment2 + os.path.basename(audio_file).split('.')[0] + "_" + str(file_name_counter).zfill(4) + ".wav"
  segmento.export(out_filename, format="wav")

  time_current += period
  file_name_counter += 1
  i += 1
print('Done!')

In [ ]:
# Corroborar los archivos generados 
from glob import glob 

In [ ]:
pathSegment2 = pathSegment + person[2] + "/"
files = glob(pathSegment2+"*wav")
files[:2]

['./drive/MyDrive/Master_Program_CC/AprendizajeAutomatico/Examen_final/RawData/data_segmentada/juan-carlos/data_original1_1395.wav',
 './drive/MyDrive/Master_Program_CC/AprendizajeAutomatico/Examen_final/RawData/data_segmentada/juan-carlos/data_original1_1396.wav']

In [ ]:
files2 = sorted(files)
print(len(files2))
print(files2[0])
print(files2[-1])

432
./drive/MyDrive/Master_Program_CC/AprendizajeAutomatico/Examen_final/RawData/data_segmentada/juan-carlos/data_original1_1395.wav
./drive/MyDrive/Master_Program_CC/AprendizajeAutomatico/Examen_final/RawData/data_segmentada/juan-carlos/data_original1_2142.wav


In [ ]:
# Look for certain segmented files 
import os 
path_aux = root_path+"data_segmentada/"
lista_archivos = glob(path_aux+"*wav")
for archivo in lista_archivos:
  file_name = os.path.basename(archivo).split('_')[-1]
  #print(file_name)
  if file_name[:3]=='s04':
    print(archivo)

./drive/MyDrive/Master_Program_CC/AprendizajeAutomatico/Examen_final/RawData/data_segmentada/data_original1_0376_s04.wav
./drive/MyDrive/Master_Program_CC/AprendizajeAutomatico/Examen_final/RawData/data_segmentada/data_original1_0177_s04.wav


## Forma alternativa de segmentar
Ojo: no logra procesar para archivos wav de entrada muy grandes

In [ ]:
pip install librosa soundfile

In [ ]:
# Process a file 
import librosa, librosa.display
import soundfile as sf
import numpy as np
import os

pathSegment = root_path + "data_segmentada/" # output path 

# First load the file
audio, sr = librosa.load(audio_file)

# Period: 15s 
buffer = 15 * sr

samples_total = len(audio)
samples_wrote = 0
counter = 855 #1

while samples_wrote < samples_total:

    #check if the buffer is not exceeding total samples 
    if buffer > (samples_total - samples_wrote):
        buffer = samples_total - samples_wrote

    block = audio[samples_wrote : (samples_wrote + buffer)]
    out_filename = pathSegment + os.path.basename(audio_file).split('.')[0] + "_" + str(counter).zfill(4) + ".wav"

    sf.write(out_filename, block, sr)

    counter += 1
    samples_wrote += buffer

In [ ]:
# Process many files at once 
import librosa, librosa.display
import soundfile as sf
import numpy as np
import os

pathAudio = root_path
files = librosa.util.find_files(pathAudio, ext=['wav'])
files = np.asarray(files)

pathSegment = root_path + "data_segmentada/"
if not os.path.exists(pathSegment):
    os.makedirs(pathSegment)

for file in files: 

    # First load the file
    audio, sr = librosa.load(file)

    # Get number of samples for 2 seconds; replace 2 by any number
    buffer = 15 * sr

    samples_total = len(audio)
    samples_wrote = 0
    counter = 855 #1

    while samples_wrote < samples_total:

        #check if the buffer is not exceeding total samples 
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote

        block = audio[samples_wrote : (samples_wrote + buffer)]
        out_filename = pathSegment + os.path.basename(file).split('.')[0] + "_" + str(counter).zfill(4) + ".wav"

        sf.write(out_filename, block, sr)

        counter += 1
        samples_wrote += buffer